In [1]:
from sgembeddings import data, graphoperations
import keras

Using TensorFlow backend.


In [2]:
import stellargraph as sg

node_data, G = data.get_graph()


In [3]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5278

 Node types:
  paper: [2708]
    Edge types: paper-cities->paper

 Edge types:
    paper-cities->paper: [5278]



In [4]:
train_gen = graphoperations.create_train_gen(G)

Running GraphSAGELinkGenerator with an estimated 542 batches generated on the fly per epoch.


In [5]:
graphsage = graphoperations.create_graphsage(train_gen)

In [6]:
x_inp, x_out, model = graphoperations.create_model(graphsage)

Instructions for updating:
Colocations handled automatically by placer.
link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [ ]:
history = graphoperations.train_model(model, train_gen)

Instructions for updating:
Use tf.cast instead.
Epoch 1/4
542/542 [==============================] - 54s 99ms/step - loss: 0.5677 - binary_accuracy: 0.7451
Epoch 2/4
542/542 [==============================] - 53s 98ms/step - loss: 0.5405 - binary_accuracy: 0.7808
Epoch 3/4
542/542 [==============================] - 53s 98ms/step - loss: 0.5382 - binary_accuracy: 0.7854
Epoch 4/4
403/542 [=====================>........] - ETA: 13s - loss: 0.5368 - binary_accuracy: 0.7885

The inputs and outputs to the node pair classifier model are 2 sets of identical inputs and outputs.

In [ ]:
x_inp

In [ ]:
x_out

We take the odd elements of the input and the first element of the output, and create a new embedding model from these.

In [ ]:
x_inp_src = x_inp[0::2]
x_out_src = x_out[0]
embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)


In [ ]:
from stellargraph.mapper import GraphSAGENodeGenerator
from sgembeddings import config


node_ids = node_data.index
node_gen = GraphSAGENodeGenerator(G, config.BATCH_SIZE, config.NUM_SAMPLES).flow(node_ids)


In [ ]:
emb = embedding_model.predict_generator(node_gen, workers=4, verbose=1)
node_embeddings = emb[:,0,:]


In [ ]:
node_embeddings.shape